In [ ]:
# ==============================================================================
# PROJECT: ENTERPRISE AUDIT AUTOMATION (ULTIMATE CORPORATE EDITION)
# ==============================================================================

# 1. CÀI ĐẶT THƯ VIỆN
!pip install faker fpdf xlsxwriter pandas matplotlib seaborn python-docx

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from faker import Faker
from fpdf import FPDF
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from datetime import datetime, timedelta
import os

# 1. SETUP ENVIRONMENT
if not os.path.exists('audit_charts'): os.makedirs('audit_charts')
fake = Faker('en_US')
np.random.seed(2025)

print("🚀 KICKOFF: Generating 12,000 Enterprise Transactions...")

# ==============================================================================
# PHẦN 1: DATA GENERATION (12K Rows - English - Strict Logic)
# ==============================================================================

expense_logic = {
    'Travel': {'merchants': ['Uber', 'Lyft', 'Delta Air', 'United', 'Hertz', 'Amtrak'], 'range': (30, 1500)},
    'Meals': {'merchants': ['Starbucks', 'Chipotle', 'Ruths Chris', 'Dominos', 'Panera', 'Local Diner'], 'range': (15, 400)},
    'Lodging': {'merchants': ['Marriott', 'Hilton', 'Airbnb', 'Hyatt', 'Holiday Inn'], 'range': (200, 3000)},
    'IT Hardware': {'merchants': ['Apple Store', 'Dell', 'CDW', 'BestBuy', 'Newegg'], 'range': (100, 4000)},
    'Software': {'merchants': ['AWS', 'Salesforce', 'Slack', 'Zoom', 'Atlassian'], 'range': (50, 5000)},
    'Office': {'merchants': ['Staples', 'Office Depot', 'Uline', 'Amazon Biz'], 'range': (20, 1000)}
}

employees = [f'EMP{i:03d}' for i in range(1, 151)] # 150 nhân viên
data = []

# Sinh 12.000 dòng sạch
for _ in range(12000):
    date = fake.date_between(start_date='-1y', end_date='today')
    emp = random.choice(employees)
    cat = random.choice(list(expense_logic.keys()))
    merch = random.choice(expense_logic[cat]['merchants'])
    amount = round(random.uniform(expense_logic[cat]['range'][0], expense_logic[cat]['range'][1]), 2)
    desc = f"{cat} expense at {merch}"
    data.append([date, emp, cat, amount, merch, desc])

df = pd.DataFrame(data, columns=['Date', 'EmployeeID', 'Category', 'Amount', 'Merchant', 'Description'])
df['Date'] = pd.to_datetime(df['Date'])

# --- CẤY LỖI (FRAUD INJECTION) ---
print("💉 Injecting Advanced Fraud Patterns...")

# 1. Weekend Warriors (150 vụ)
for i in range(150):
    df.loc[i, 'Date'] = pd.Timestamp('2023-11-11') # Saturday
    df.loc[i, 'Category'] = 'Meals'
    df.loc[i, 'Description'] = 'Personal Weekend Dinner'
    df.loc[i, 'Merchant'] = 'The Night Club'

# 2. Duplicate Claims (60 cặp = 120 dòng)
dupes = df.iloc[200:260].copy()
df = pd.concat([df, dupes], ignore_index=True)

# 3. Split Transactions (40 vụ)
for i in range(40):
    split_date = pd.Timestamp('2023-12-01')
    df.loc[len(df)] = [split_date, 'EMP099', 'IT Hardware', 4900.00, 'Apple Store', 'Macbook Pro Part A']
    df.loc[len(df)] = [split_date, 'EMP099', 'IT Hardware', 4900.00, 'Apple Store', 'Macbook Pro Part B']

# >>> EXPORT RAW DATA <<<
df.to_csv('Raw_Expense_Data.csv', index=False)
print("✅ File 1: Raw_Expense_Data.csv (Created)")

# ==============================================================================
# PHẦN 2: AUDIT LOGIC
# ==============================================================================
print("🔍 Running Audit Engine...")

# Rule 1: Weekend
df['Is_Weekend'] = df['Date'].dt.weekday >= 5
weekend_fraud = df[(df['Is_Weekend'] == True) & (~df['Category'].isin(['Travel', 'Lodging']))]

# Rule 2: Duplicates
duplicate_fraud = df[df.duplicated(subset=['Date', 'EmployeeID', 'Amount', 'Merchant'], keep=False)]

# Rule 3: High Value / Split Limit
limit_fraud = df[df['Amount'] > 4500]

df['Audit_Finding'] = 'Normal'
df.loc[weekend_fraud.index, 'Audit_Finding'] = 'Weekend Violation'
df.loc[duplicate_fraud.index, 'Audit_Finding'] = 'Duplicate Claim'
df.loc[limit_fraud.index, 'Audit_Finding'] = 'High Value / Split Risk'

fraud_only = df[df['Audit_Finding'] != 'Normal'].copy().sort_values(by='Amount', ascending=False)

# ==============================================================================
# PHẦN 3: EXCEL REPORT (6 NATIVE CHARTS)
# ==============================================================================
print("📊 Generating Excel Dashboard...")
excel_file = 'Audit_Report_Dashboard.xlsx'
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
workbook = writer.book

# Summary Tables
s_type = df[df['Audit_Finding'] != 'Normal']['Audit_Finding'].value_counts().reset_index()
s_type.columns = ['Type', 'Count']
s_emp = fraud_only.groupby('EmployeeID')['Amount'].sum().nlargest(5).reset_index()
s_emp.columns = ['Employee', 'Risk_Value']
s_cat = fraud_only.groupby('Category')['Amount'].sum().reset_index()
s_cat.columns = ['Category', 'Risk_Value']
s_date = fraud_only.groupby(fraud_only['Date'].dt.to_period('M'))['Amount'].sum().reset_index()
s_date['Date'] = s_date['Date'].astype(str)

# Write Sheets
s_type.to_excel(writer, sheet_name='Dashboard', index=False, startrow=0, startcol=0)
s_emp.to_excel(writer, sheet_name='Dashboard', index=False, startrow=0, startcol=4)
s_cat.to_excel(writer, sheet_name='Dashboard', index=False, startrow=0, startcol=8)
s_date.to_excel(writer, sheet_name='Dashboard', index=False, startrow=0, startcol=12)
df.to_excel(writer, sheet_name='Detailed_Audit_Log', index=False)

ws_dash = writer.sheets['Dashboard']
ws_det = writer.sheets['Detailed_Audit_Log']

# --- 6 NATIVE EXCEL CHARTS ---
def add_chart(type, name, cat, val, cell, title, color):
    chart = workbook.add_chart({'type': type})
    chart.add_series({'name': name, 'categories': cat, 'values': val, 'fill': {'color': color}})
    chart.set_title({'name': title})
    ws_dash.insert_chart(cell, chart)

add_chart('column', 'Count', '=Dashboard!$A$2:$A$5', '=Dashboard!$B$2:$B$5', 'A10', 'Violations by Type', '#C00000')
add_chart('bar', 'Risk($)', '=Dashboard!$E$2:$E$6', '=Dashboard!$F$2:$F$6', 'E10', 'Top Offenders', '#FF9900')
add_chart('doughnut', 'Risk', '=Dashboard!$I$2:$I$7', '=Dashboard!$J$2:$J$7', 'I10', 'Risk by Category', '#0000FF')

# Line Chart for Trend
chart4 = workbook.add_chart({'type': 'line'})
chart4.add_series({'name': 'Trend', 'categories': '=Dashboard!$M$2:$M$13', 'values': '=Dashboard!$N$2:$N$13', 'line': {'color': 'red', 'width': 2.25}})
chart4.set_title({'name': 'Fraud Trend'})
ws_dash.insert_chart('A26', chart4)

# Pie Chart
chart5 = workbook.add_chart({'type': 'pie'})
chart5.add_series({'categories': '=Dashboard!$A$2:$A$5', 'values': '=Dashboard!$B$2:$B$5'})
chart5.set_title({'name': 'Violation Mix'})
ws_dash.insert_chart('E26', chart5)

# Area Chart
chart6 = workbook.add_chart({'type': 'area'})
chart6.add_series({'categories': '=Dashboard!$M$2:$M$13', 'values': '=Dashboard!$N$2:$N$13', 'fill': {'color': '#FFCCCC'}})
chart6.set_title({'name': 'Cumulative Exposure'})
ws_dash.insert_chart('I26', chart6)

# Conditional Formatting (Red for Fraud)
red_fmt = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
ws_det.conditional_format('G2:G15000', {'type': 'formula', 'criteria': '=$G2<>"Normal"', 'format': red_fmt})
ws_det.set_column('A:G', 18)

writer.close()
print(f"✅ File 2: {excel_file} (6 Charts)")

# ==============================================================================
# PHẦN 4: VISUALIZATION FOR REPORT
# ==============================================================================
print("🎨 Generating Charts for PDF/Word...")
sns.set_style("whitegrid")
plt.figure(figsize=(10, 5)); sns.countplot(y='Audit_Finding', data=fraud_only, palette='Reds_r'); plt.title('Frequency of Audit Findings'); plt.tight_layout(); plt.savefig('audit_charts/c1.png'); plt.close()
plt.figure(figsize=(10, 5)); top_emp = fraud_only.groupby('EmployeeID')['Amount'].sum().nlargest(5); sns.barplot(x=top_emp.values, y=top_emp.index, palette='Oranges_r'); plt.title('Top Risk Employees ($)'); plt.tight_layout(); plt.savefig('audit_charts/c2.png'); plt.close()
plt.figure(figsize=(10, 5)); daily = fraud_only.groupby('Date')['Amount'].sum().reset_index(); sns.lineplot(x='Date', y='Amount', data=daily, color='red'); plt.title('Risk Trend'); plt.tight_layout(); plt.savefig('audit_charts/c3.png'); plt.close()
plt.figure(figsize=(10, 5)); sns.histplot(fraud_only['Amount'], bins=20, color='navy'); plt.title('Fraud Amount Distribution'); plt.tight_layout(); plt.savefig('audit_charts/c4.png'); plt.close()

# ==============================================================================
# PHẦN 5: WORD REPORT (EDITABLE)
# ==============================================================================
print("📝 Generating Word Report...")
doc = Document()
style = doc.styles['Normal']
style.font.name = 'Arial'
style.font.size = Pt(12)

# Cover
doc.add_heading('INTERNAL AUDIT REPORT', 0)
doc.add_paragraph(f'Date: {datetime.now().strftime("%B %d, %Y")}\nScope: 12,000 Transactions')
doc.add_heading('1. Executive Summary', 1)
doc.add_paragraph("The audit identified significant control deficiencies. A total of 12,000 transactions were scanned, revealing patterns of weekend spending abuse, duplicate claims, and structuring of high-value payments.")
doc.add_heading('2. Visual Analysis', 1)
doc.add_picture('audit_charts/c1.png', width=Inches(6))
doc.add_page_break()

# Table
doc.add_heading('3. Detailed Findings', level=1)
table = doc.add_table(rows=1, cols=5)
table.style = 'Table Grid'
hdr = table.rows[0].cells
hdr[0].text = 'Date'; hdr[1].text = 'Emp'; hdr[2].text = 'Category'; hdr[3].text = 'Amount'; hdr[4].text = 'Finding'

for i, row in fraud_only.head(50).iterrows(): # Add 50 rows to Word
    row_cells = table.add_row().cells
    row_cells[0].text = str(row['Date'].date())
    row_cells[1].text = row['EmployeeID']
    row_cells[2].text = row['Category']
    row_cells[3].text = f"${row['Amount']:,.0f}"
    row_cells[4].text = row['Audit_Finding']

doc.save('Audit_Report_Editable.docx')
print("✅ File 3: Audit_Report_Editable.docx")

# ==============================================================================
# PHẦN 6: PDF REPORT (10 PAGES - DETAILED TEXT)
# ==============================================================================
print("📄 Generating 10-Page PDF...")

class PDF(FPDF):
    def header(self):
        if self.page_no() > 1:
            self.set_font('Arial', 'B', 10)
            self.set_text_color(100)
            self.cell(0, 10, 'CONFIDENTIAL - INTERNAL AUDIT REPORT', 0, 0, 'R')
            self.ln(15)
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 16) # Header Font 16
        self.set_text_color(0) # Black
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)

    def body_text(self, text):
        self.set_font('Arial', '', 13) # Body Font 13
        self.set_text_color(0) # Black
        self.multi_cell(0, 8, text) # Line height 8
        self.ln(5)

pdf = PDF()
pdf.set_auto_page_break(auto=True, margin=20)

# PAGE 1: COVER PAGE
pdf.add_page()
pdf.ln(80)
pdf.set_font('Arial', 'B', 36)
pdf.cell(0, 20, 'INTERNAL AUDIT REPORT', 0, 1, 'C')
pdf.set_font('Arial', '', 24)
pdf.cell(0, 15, 'AUTOMATED EXPENSE CONTROL', 0, 1, 'C')
pdf.ln(30)
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, f'Report Date: {datetime.now().strftime("%B %d, %Y")}', 0, 1, 'C')
pdf.cell(0, 10, 'Prepared by: Internal Audit Department', 0, 1, 'C')
pdf.cell(0, 10, 'Distribution: CFO, Audit Committee', 0, 1, 'C')

# PAGE 2: TOC & AUDIT PLANNING (Mùi Audit)
pdf.add_page()
pdf.chapter_title('1. AUDIT PLANNING & SCOPE')

plan_text = """
OBJECTIVE:
The primary objective of this audit is to assess the effectiveness of internal controls regarding Travel & Entertainment (T&E) expenses. We aim to identify instances of policy non-compliance, potential fraud, and financial leakage within the organization.

SCOPE OF AUDIT:
- Audit Period: Last 12 Months rolling.
- Data Volume: 12,000+ Transactions processed.
- Departments Covered: Sales, Marketing, IT, Operations.
- Risk Areas: Duplicate Payments, Split Transactions, Weekend Spend.

AUDIT METHODOLOGY:
We utilized a Continuous Monitoring approach powered by Python automation.
1. Data Extraction: Raw data was ingested from the ERP system.
2. Data Integrity Check: Validated for completeness and accuracy.
3. Rule-Based Testing: Applied predefined logic (e.g., Weekend checks).
4. Statistical Analysis: Utilized outlier detection for high-value claims.
"""
pdf.body_text(plan_text)

pdf.ln(10)
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, 'TABLE OF CONTENTS', 0, 1)
pdf.set_font('Arial', '', 13)
pdf.cell(0, 8, '1. Audit Planning & Scope ................................... Page 2', 0, 1)
pdf.cell(0, 8, '2. Executive Summary ........................................... Page 3', 0, 1)
pdf.cell(0, 8, '3. Data Visualization & EDA ................................ Page 4', 0, 1)
pdf.cell(0, 8, '4. Insights & Recommendations ........................ Page 6', 0, 1)
pdf.cell(0, 8, '5. Detailed Findings Log ...................................... Page 7', 0, 1)

# PAGE 3: EXECUTIVE SUMMARY
pdf.add_page()
pdf.chapter_title('2. EXECUTIVE SUMMARY')
risk_val = fraud_only['Amount'].sum()
risk_rate = (len(fraud_only) / len(df)) * 100
text_p3 = f"""Based on our automated review of {len(df):,} transactions, the Internal Audit team has identified significant control deficiencies that require immediate management attention.

SUMMARY OF FINDINGS:
The audit flagged {len(fraud_only)} transactions as high-risk or non-compliant.
- Total Potential Financial Loss: ${risk_val:,.2f}
- Overall Error Rate: {risk_rate:.2f}%

KEY RISK AREAS IDENTIFIED:
1. STRUCTURING (SPLIT TRANSACTIONS):
We detected multiple instances where employees split large invoices into smaller amounts to bypass approval thresholds. This indicates a deliberate attempt to circumvent controls.

2. DUPLICATE PAYMENTS:
A high volume of duplicate claims was identified. This suggests that the current expense reporting system lacks a preventive control to flag duplicates at the time of entry.

3. WEEKEND SPENDING:
A pattern of excessive spending on weekends (Saturday/Sunday) within the 'Meals' category was observed, which may violate the company's policy on personal expenses.

CONCLUSION:
The control environment requires strengthening. We recommend immediate recovery of identified funds and a review of the expense approval policy."""
pdf.body_text(text_p3)

# PAGE 4-5: VISUALS
pdf.add_page()
pdf.chapter_title('3. DATA VISUALIZATION (PART 1)')
pdf.body_text("The following charts illustrate the frequency of different violation types and identifying the top personnel involved in high-risk activities. These visuals highlight areas requiring immediate attention.")
pdf.image('audit_charts/c1.png', x=15, w=180)
pdf.ln(5)
pdf.body_text("Figure 1: Breakdown of violations showing 'Weekend Violation' as the most frequent issue.")
pdf.ln(5)
pdf.image('audit_charts/c2.png', x=15, w=180)
pdf.body_text("Figure 2: Top 5 employees by risk value. These individuals account for a significant portion of the total potential loss.")

pdf.add_page()
pdf.chapter_title('3. DATA VISUALIZATION (PART 2)')
pdf.body_text("Trend analysis allows us to identify seasonal patterns in fraudulent behavior, while distribution analysis highlights financial outliers in the dataset.")
pdf.image('audit_charts/c3.png', x=15, w=180)
pdf.ln(5)
pdf.body_text("Figure 3: Timeline of fraud exposure indicating spikes at month-end.")
pdf.ln(5)
pdf.image('audit_charts/c4.png', x=15, w=180)
pdf.body_text("Figure 4: Distribution of fraud amounts showing clusters of high-value transactions.")

# PAGE 6: INSIGHTS
pdf.add_page()
pdf.chapter_title('4. DEEP DIVE INSIGHTS & RECOMMENDATIONS')
text_p6 = """DETAILED INSIGHTS:
Our analysis reveals that the majority of financial leakage is not due to random error, but rather specific behavioral patterns. The 'Structuring' of payments is particularly concerning as it indicates intent. Furthermore, the high number of duplicate payments suggests that employees are either submitting receipts across different reporting periods or the system is failing to catch user errors.

STRATEGIC RECOMMENDATIONS:

1. IMMEDIATE REMEDIATION (Priority: High):
   - Initiate a formal investigation into the Top 5 employees identified in this report.
   - Review and recover the identified duplicate payments immediately.
   - Suspend corporate cards for employees with repeated 'Split Transaction' violations.

2. PROCESS IMPROVEMENT (Priority: Medium):
   - Implement a system 'Hard Block' for any expense claim submitted for a weekend date unless accompanied by approved travel documentation.
   - Update the expense policy to explicitly define 'Structuring' as a termination-level offense.

3. TECHNOLOGY ENHANCEMENT (Priority: Low):
   - Deploy this automated Python audit script as a continuous monitoring tool to run on a weekly basis, shifting from retrospective audit to real-time detection.
"""
pdf.body_text(text_p6)

# PAGE 7-11: DETAILED TABLE
pdf.add_page()
pdf.chapter_title('5. DETAILED AUDIT FINDINGS')
pdf.body_text("The following table lists the high-risk transactions identified for further review. Please cross-reference with original receipts.")
pdf.set_font('Arial', 'B', 10)
pdf.set_fill_color(220, 220, 220)
pdf.cell(30, 8, 'Date', 1, 0, 'C', 1)
pdf.cell(25, 8, 'Emp ID', 1, 0, 'C', 1)
pdf.cell(40, 8, 'Category', 1, 0, 'C', 1)
pdf.cell(30, 8, 'Amount', 1, 0, 'C', 1)
pdf.cell(65, 8, 'Finding', 1, 1, 'C', 1)

# Print 400 rows to fill pages
pdf.set_font('Arial', '', 9)
for i, row in fraud_only.head(400).iterrows():
    pdf.cell(30, 7, row['Date'].strftime("%Y-%m-%d"), 1)
    pdf.cell(25, 7, row['EmployeeID'], 1)
    pdf.cell(40, 7, row['Category'], 1)
    pdf.cell(30, 7, f"${row['Amount']:,.0f}", 1)
    pdf.cell(65, 7, row['Audit_Finding'], 1, 1)

pdf.output("Audit_Report_Final_Professional.pdf")
print("✅ File 4: Audit_Report_Final_Professional.pdf (10+ Pages)")
print("\n🎉 MISSION ACCOMPLISHED! All 4 Files Generated.")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.8 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=dd76933b736967031fc377e2eb2873952b2dee57a6a5ef84ab0e84aee29bc2ff
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf
🚀 KICKOFF: Generating 12,000 Enterprise Transactions...
💉 Injecting Advanced Fraud Patterns...
✅ File 1: Raw_Expense_Data.csv (Created)
🔍 Running Audit Engine...
📊 Generating Excel Dashboard...
✅ File 2: Audit_Report_Dashboard.xlsx (6 Charts)
🎨 Generating Charts for PDF/Word...


/tmp/ipython-input-1717817838.py:173: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  plt.figure(figsize=(10, 5)); sns.countplot(y='Audit_Finding', data=fraud_only, palette='Reds_r'); plt.title('Frequency of Audit Findings'); plt.tight_layout(); plt.savefig('audit_charts/c1.png'); plt.close()
/tmp/ipython-input-1717817838.py:174: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  plt.figure(figsize=(10, 5)); top_emp = fraud_only.groupby('EmployeeID')['Amount'].sum().nlargest(5); sns.barplot(x=top_emp.values, y=top_emp.index, palette='Oranges_r'); plt.title('Top Risk Employees ($)'); plt.tight_layout(); plt.savefig('audit_charts/c2.png'); plt.close()


📝 Generating Word Report...
✅ File 3: Audit_Report_Editable.docx
📄 Generating 10-Page PDF...
✅ File 4: Audit_Report_Final_Professional.pdf (10+ Pages)

🎉 MISSION ACCOMPLISHED! All 4 Files Generated.
